In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from lightgbm import LGBMClassifier
import optuna
from optuna.samplers import TPESampler # this is used to sample the hyperparameters

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns',600)
pd.set_option('display.max_rows',450)

sns.set(style="whitegrid")